<a href="https://colab.research.google.com/github/LuisDavid999/UDEA-ai4eng-20251-Pruebas-Saber-Pro-Colombia/blob/main/05_XGBoostregiones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Extreme Gradient Boosting**

XGBoost (eXtreme Gradient Boosting) es una biblioteca de machine learning distribuida y de código abierto que emplea decision trees potenciados por gradiente, un algoritmo de impulso del aprendizaje supervisado que emplea el descenso de gradiente. Es conocido por su velocidad, eficiencia y capacidad de escalar bien con grandes conjuntos de datos.

## **Cargar los archivos trainregiones.csv y testregiones.csv**






In [8]:
!pip install xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost.core")
import matplotlib.pyplot as plt

In [9]:
train = pd.read_csv('trainregiones.csv')
test = pd.read_csv('testregiones.csv')
train.shape, test.shape

((692500, 33), (296786, 25))

In [10]:
X = train.drop('RENDIMIENTO_GLOBAL', axis = 1)
y = train['RENDIMIENTO_GLOBAL'].values
X_test = test.drop(['ID'],axis=1)

## **Particionar los datos**


In [11]:
test_size = 0.3

Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=test_size)
print (Xtr.shape, Xts.shape)

(484750, 32) (207750, 32)


In [18]:
estimator = XGBClassifier(n_estimators=200, max_depth=5, learning_rate=0.1, objective='multi:softprob')

In [13]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.3, 0.5]
}

In [14]:
# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(Xtr, ytr)

# Get the best estimator
best_estimator = grid_search.best_estimator_

print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy found: ", grid_search.best_score_)

Best parameters found:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Best accuracy found:  0.41320061833450583


## **Entrenar el modelo**

In [19]:
estimator.fit(Xtr, ytr)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

## **Evaluar el modelo**

In [20]:
estimator.score(Xtr, ytr), estimator.score(Xts,yts)


(0.4235605982465188, 0.41316004813477736)

Con el modelo ahora entrenado, vamos a preparar el archivo de test para hacer las predicciones

In [21]:
Xtest = test.drop('ID', axis = 1)
id = test['ID'].values
predicts = estimator.predict(Xtest)
predicts = np.round(predicts)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:ESTU_PRGM_ACADEMICO: object, ESTU_PRGM_DEPARTAMENTO: object, ESTU_VALORMATRICULAUNIVERSIDAD: object, ESTU_HORASSEMANATRABAJA: object, FAMI_ESTRATOVIVIENDA: object, FAMI_TIENEINTERNET: object, FAMI_EDUCACIONPADRE: object, FAMI_TIENELAVADORA: object, FAMI_TIENEAUTOMOVIL: object, ESTU_PRIVADO_LIBERTAD: object, ESTU_PAGOMATRICULAPROPIO: object, FAMI_TIENECOMPUTADOR: object, FAMI_TIENEINTERNET.1: object, FAMI_EDUCACIONMADRE: object

## **Formatear los resultados obtenidos para Kaggle**

In [ ]:
id_test = pd.DataFrame(test["ID"])
rendimiento_global = ['bajo', 'medio-bajo', 'medio-alto', 'alto']
results = [rendimiento_global[int(i)] for i in predicts]
id_test['RENDIMIENTO_GLOBAL'] = pd.Series(results, name='RENDIMIENTO_GLOBAL')

id_test.shape
id_test.head()

id_test.to_csv('submission.csv', index = False) #Descargar Submission para Kaggle